In [8]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Data Loading

In [71]:
df_ticker = pd.read_csv("../data/stock/MSFT.csv")
df_ticker["Date"] = pd.to_datetime(df_ticker["Date"], utc=True)
df_ticker["ts"] = pd.to_datetime(df_ticker["Date"].dt.date)
df_ticker.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Goal1,Goal2,ts
0,2022-12-28 05:00:00+00:00,233.531667,236.321549,230.850227,231.205124,17457100,0.0,0.0,0,0,2022-12-28
1,2022-12-29 05:00:00+00:00,232.309233,238.490349,232.309233,237.593246,19770700,0.0,0.0,1,0,2022-12-29
2,2022-12-30 05:00:00+00:00,234.832929,236.558120,233.304900,236.420105,21938500,0.0,0.0,0,0,2022-12-30
3,2023-01-03 05:00:00+00:00,239.633899,242.266045,234.034415,236.183517,25740000,0.0,0.0,0,0,2023-01-03
4,2023-01-04 05:00:00+00:00,228.986992,229.568624,222.756597,225.852081,50623400,0.0,0.0,1,1,2023-01-04


In [43]:
df_ticker.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   Date          503 non-null    datetime64[ns, UTC]
 1   Open          503 non-null    float64            
 2   High          503 non-null    float64            
 3   Low           503 non-null    float64            
 4   Close         503 non-null    float64            
 5   Volume        503 non-null    int64              
 6   Dividends     503 non-null    float64            
 7   Stock Splits  503 non-null    float64            
 8   Goal1         503 non-null    int64              
 9   Goal2         503 non-null    int64              
 10  ts            503 non-null    datetime64[ns]     
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), float64(6), int64(3)
memory usage: 43.4 KB


In [76]:
df_ticker["ts"].min()

Timestamp('2022-12-28 00:00:00')

In [47]:
df_sentiment = pd.read_csv("../data/sentiment.csv", parse_dates=["publication_date"])
df_sentiment["ts"] = pd.to_datetime(df_sentiment["publication_date"].dt.date)
df_sentiment.head()

,id,source,publication_date,company,sentiment,confidence,ts
0,5a618a2daa89d751b75a0c76732cca3b,CNN,2024-12-19 20:32:19.900,APPLE,NEGATIVE,0.8,2024-12-19
1,89a222209eced5c8fbb71670a8817219,CNN,2024-12-03 12:33:00.000,OTHER,NEUTRAL,0.8,2024-12-03
2,37a292b603e0313952c8fd6b623233ca,CNN,2024-12-03 00:50:00.000,OTHER,POSITIVE,0.9,2024-12-03
3,c947aa040c36f2621b7231012e451594,CNN,2024-12-03 05:55:00.000,APPLE,POSITIVE,0.9,2024-12-03
4,94d4c11e27bba1525f2d26af836e87a3,CNN,2024-12-03 03:55:00.000,APPLE,POSITIVE,0.9,2024-12-03


In [48]:
df_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 543 entries, 0 to 542
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                543 non-null    object        
 1   source            543 non-null    object        
 2   publication_date  543 non-null    datetime64[ns]
 3   company           543 non-null    object        
 4   sentiment         543 non-null    object        
 5   confidence        543 non-null    float64       
 6   ts                543 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(4)
memory usage: 29.8+ KB


In [80]:
df_sentiment["source"].value_counts()

source
BBC        195
Reuters    189
CNN        159
Name: count, dtype: int64

## Visualization

In [73]:
def calc_value(row):
    if row["sentiment"] == "POSITIVE":
        return 1
    if row["sentiment"] == "NEGATIVE":
        return -1
    
    return 0

df_sentiment_subset = df_sentiment[(df_sentiment["company"] == "MICROSOFT") & (df_sentiment["ts"] >= df_ticker["ts"].min())]
df_sentiment_subset = df_sentiment_subset.copy().sort_values("publication_date")
df_sentiment_subset["value"] = df_sentiment_subset.apply(calc_value, axis=1)
df_sentiment_subset.head()

,id,source,publication_date,company,sentiment,confidence,ts,value
103,f3199d83cd250556c543a75ad58bcf22,CNN,2023-10-02 19:20:16,MICROSOFT,NEGATIVE,0.8,2023-10-02,-1
477,d5158f45abb3ea77ebce38f860b9de67,BBC,2023-10-05 09:48:09,MICROSOFT,NEUTRAL,0.8,2023-10-05,0
102,76db0077def5063ff4e573e5ece8f302,CNN,2023-10-05 10:38:58,MICROSOFT,NEUTRAL,0.8,2023-10-05,0
99,1b8b184c72ab28438c3ebf1b1fbeb0fa,CNN,2023-10-11 21:22:40,MICROSOFT,NEUTRAL,0.8,2023-10-11,0
101,e082c036aa34ac8ae55f5f124e15f5f4,CNN,2023-10-12 16:33:45,MICROSOFT,NEUTRAL,0.8,2023-10-12,0


In [74]:
signals = df_sentiment_subset.groupby("ts")["value"].mean().reset_index()
signals = signals.merge(df_ticker[["ts", "Close"]], on="ts", how="left")
signals.head()

,ts,value,Close
0,2023-10-02,-1.0,319.402496
1,2023-10-05,0.0,316.980682
2,2023-10-11,0.0,329.943451
3,2023-10-12,0.0,328.692810
4,2023-10-24,1.0,328.067505


In [75]:
fig = px.line(df_ticker, x="Date", y="Close")
fig.add_trace(go.Scatter(mode="markers", x=signals[signals["value"] < 0.0]["ts"], y=signals[signals["value"] < 0.0]["Close"], fillcolor="red"))
fig.add_trace(go.Scatter(mode="markers", x=signals[signals["value"] > 0.0]["ts"], y=signals[signals["value"] > 0.0]["Close"], fillcolor="green"))
fig.show()